In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../src')
from preprocess import get_train_test

In [2]:
train, test = get_train_test(args=0)
train.head()

/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,registor_number,user_id,sex,age,distance_to_the_store,start_at__hour,target,month,day,week
0,0,0,0,37.0,1.0,13,0,11,1,0
1,1,1,0,62.0,7.0,14,0,11,1,0
2,2,2,0,32.0,5.0,12,0,11,1,0
3,3,3,1,32.0,8.0,16,0,11,1,0
4,4,4,0,67.0,2.0,14,0,11,1,0


In [3]:
train.dtypes

registor_number          category
user_id                  category
sex                      category
age                       float16
distance_to_the_store     float16
start_at__hour              int64
target                      uint8
month                    category
day                      category
week                     category
dtype: object

In [14]:
train['lag_target'] = train.groupby('user_id').target.shift(1)
train.loc[train['user_id']==5]

,registor_number,user_id,sex,age,distance_to_the_store,start_at__hour,target,month,day,week,lag_target
5,5,5,0,42.0,1.0,17,0,11,1,0,NaN
2554,48,5,0,42.0,1.0,16,0,11,2,1,0.0
3587,1,5,0,42.0,1.0,10,0,11,3,2,0.0
5512,164,5,0,42.0,1.0,16,0,11,4,3,0.0
12067,65,5,0,42.0,1.0,17,0,11,9,1,0.0
12119,182,5,0,42.0,1.0,18,0,11,9,1,0.0
13118,175,5,0,42.0,1.0,17,0,11,10,2,0.0
14362,190,5,0,42.0,1.0,17,0,11,11,3,0.0
15699,174,5,0,42.0,1.0,17,0,11,12,4,0.0
16958,150,5,0,42.0,1.0,16,0,11,13,5,0.0


In [4]:
DATA_PATH = '../input/'
import os

data = pd.read_csv(os.path.join(DATA_PATH, "session.csv"))
test_session = pd.read_csv(os.path.join(DATA_PATH, "test_session.csv"))


In [5]:
data['test'] = data['session_id'].isin(test_session['session_id'])
data[data['test'] == True]

,session_id,registor_number,user_id,sex,age,distance_to_the_store,start_at__date,start_at__hour,test
2,000280d2156d179a597167b269a69e5af556725eeaabae...,1016,8da8267f34ec7ee3f3b800b089b7e27c16fe8cf5a7e36b...,女性,35~39,17,2022-01-25,13,True
9,00055a0719ae8afefe0a785d4099a98da414b72345b2d9...,1051,79f9d3fa4006cb284677e95194543a3ccfe3db55cf5294...,女性,65~69,3,2022-01-12,21,True
17,0009065eec287b1cedc44425b2b53c82aba347bf9966be...,1117,8b2894cef5e1de80f0b13cc33655f9bf5d5442da3d3aac...,女性,65~69,5,2022-01-16,13,True
27,000efa1223901c67e9abc8030b55218240174b6c793289...,1077,c5bd0a9e3958a1218f6d1b9817f83e7bd02ec93151dc14...,男性,45~49,1,2022-01-25,19,True
30,001019dafd085898d4bcd7631468af2b3ccb6120b71bac...,1099,2ed875b8f5456649d34849ede7cff795513d853c14dc0e...,女性,75~79,3,2022-01-11,11,True
...,...,...,...,...,...,...,...,...,...
109135,fff17a787eadeac8c78421e7fce6f3b918091c6c550be7...,1123,6fb02f2a89a26f6c3f1a317523360782e49a0b2cc324b5...,男性,40~44,5,2022-01-14,17,True
109138,fff2a322f01b391d1b73a9ff5bf878fc2925d26a5db9b9...,1096,dfb734472e76a9ccfb121e219c78fb8950096631aa317c...,男性,30~34,1,2022-01-17,20,True
109149,fff7476be6ba92ceac0c7830d96d92266061433992419b...,1116,4214e9d1709d0a01d045b0918208bbb89a87825b3bae5e...,女性,40~44,3,2022-01-12,10,True
109154,fffa55a2a6d0e29505b48dc6e49e07ba004a312e23fe41...,1059,c973f71eb917860aa4d69594a237b15a87442157e64cc8...,女性,50~54,26,2022-01-30,16,True


In [6]:
data['age'].unique()

array(['60~64', '55~59', '35~39', '30~34', '25~29', '50~54', '70~74',
       '65~69', '45~49', '不明', '15~19', '75~79', '40~44', nan, '20~24',
       '80~84', '85~89', '10~14', '100~', '0~4', '90~94', '5~9', '95~99'],
      dtype=object)